In [1]:

import pandas as pd
from pandas import DataFrame
from datetime import datetime
pd.set_option('display.max_rows', 3000)

import xlrd
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True

import openpyxl

import os


In [2]:
def load_data(file_name):
    names = ['items', 'unit', 'price']
#     skipDays = datetime.now().day
    
    wb = xlrd.open_workbook(file_name)
    # obtain all the sheet in the workbook
    sheets_name = wb.sheet_names()
    # print(sheets)
    
    # too less info, omit
    if len(sheets_name) <= 5:
        return
 
    # iterator sheets without the total sheet
    df_all = DataFrame()
    for i in range(len(sheets_name)-1):
    # skiprows=2 omit tow
        df = pd.read_excel(file_name, names = names, sheet_name=i, skiprows=2, 
                       skipfooter = 3, usecols= [0,1,3])
        df_all= df_all.append(df)
        
    
    # print(df_all)

    # delete Nan value and reset index
    df_all = df_all.dropna()
    df_all = df_all.reset_index(drop=True)
    
    
    df_all.sort_values(axis=0, ascending=False, inplace=True, kind='quicksort', by=['price'])
    
    # print(df_all)
    
    df_total = pd.read_excel(file_name, 
                             # usecols: use which cols as input
                             # skiprows = 1, jump title and set header to None
                   sheet_name=len(sheets_name)-1,skiprows=1, header = None, names = names, usecols = "A:C")
    # print(df_total)

    # contact active and old database together
    frames = [df_all, df_total]
    results = pd.concat(frames)
    
#     be used for debug
    print(results)
    
    # according to prices, sort value
    results.sort_values(axis=0, ascending=False, inplace=True, 
           kind='quicksort', by=['price'])
    # print(results)
    #based on item and unit to dropout duplicated vlaue
    results.drop_duplicates(['items', 'unit'], keep = 'first', inplace = True)
    
    # reset index
    results = results.reset_index(drop=True)
    
    # print(results)
    
    workbook = openpyxl.load_workbook(file_name)
    
  
#     if len(sheets_name) > skipDays + 2:
#         numPickerList = list(range(len(sheets_name)-12, len(sheets_name)-1))
#         numPickerList.extend([0, 1])
        
    for step, i in enumerate(sheets_name):
        if step == 0 or step == len(sheets_name)-3 or step == 1:
#             if step in numPickerList:
            continue

#         print(i)

        # delete workbook with sheet name
        del workbook[i]

    # print(workbook.get_sheet_names())
    workbook._active_sheet_index = 0
    workbook.save(file_name)

    # save new data into original excel sheet
    with pd.ExcelWriter(file_name, engine="openpyxl", mode = 'a') as writer:
        # write results to that file_name, name this name in: sheet_name
        # without index
        results = results.dropna()
        results.to_excel(writer, sheet_name = sheets_name[len(sheets_name)-1], header = ['品名', '单位', '单价'], index = False)
        writer.save()


In [5]:
if __name__ == '__main__':

    static_dir = "/Users/user/processing_excel/2023.04/"  
    
    any_unrecorder = True
    dir = static_dir
    
    while any_unrecorder:
  
        filename_excel = []
        folder_name = []
        any_unrecorder = False

        for root, dirs, files in os.walk(dir):
            for file in files:

                if not file.endswith('.xlsx'):
                    continue
                # print(os.path.join(root,file))
                filename_excel.append(os.path.join(root,file))

        # print(filename_excel)

        for file in filename_excel:
            print("I m processing ", file)
            load_data(file)
            
        if len(folder_name) != 0:
            temp_folder_name = folder_name.pop(0)
            dir = static_dir + str(temp_folder_name)
            any_unrecorder = True
            
    print("processing finished")

I m processing  /Users/user/processing_excel/2023.04/佳木斯司凡雨.xlsx
I m processing  /Users/user/processing_excel/2023.04/配送中心.xlsx
I m processing  /Users/user/processing_excel/2023.04/老菜馆.xlsx
I m processing  /Users/user/processing_excel/2023.04/白玛拉姆.xlsx
I m processing  /Users/user/processing_excel/2023.04/未知.xlsx
I m processing  /Users/user/processing_excel/2023.04/大芥末.xlsx
I m processing  /Users/user/processing_excel/2023.04/大吉象烧烤.xlsx
I m processing  /Users/user/processing_excel/2023.04/去茶去.xlsx
I m processing  /Users/user/processing_excel/2023.04/五常拉林.xlsx
I m processing  /Users/user/processing_excel/2023.04/群力远大.xlsx
I m processing  /Users/user/processing_excel/2023.04/人防.xlsx
I m processing  /Users/user/processing_excel/2023.04/低频报货/五常四季.xlsx
I m processing  /Users/user/processing_excel/2023.04/低频报货/新551车.xlsx
I m processing  /Users/user/processing_excel/2023.04/低频报货/209.xlsx
I m processing  /Users/user/processing_excel/2023.04/低频报货/120.xlsx
I m processing  /Users/user/processing_e

I m processing  /Users/user/processing_excel/2023.04/高频报货/商大厨房.xlsx
I m processing  /Users/user/processing_excel/2023.04/高频报货/8E3G2.xlsx
I m processing  /Users/user/processing_excel/2023.04/高频报货/大伟.xlsx
          items unit  price
36          花生米    件  325.0
28          松花蛋    件  235.0
34        黎红藤椒油    件  125.0
30          喀秋莎    件  115.0
35          喀秋莎    件  115.0
33          麻辣油    件  105.0
37       天津蒜蓉辣酱    件   80.0
5         如意水晶粉    件   72.0
32          香其酱    件   72.0
4            香油    桶   70.0
39        万人迷鲍汁    件   55.0
16          香其酱    件   55.0
24         超霸鸡汁    瓶   50.0
29         黑胡椒汁    桶   42.0
19          韭菜花    桶   38.0
13           麻椒    斤   35.0
31          BT油    瓶   35.0
1           蔓越莓    斤   30.0
10        太太乐鸡精    袋   16.0
38         鲜奶香精    桶   16.0
21          蟹黄酱    瓶   13.0
18          葡萄酒    桶   11.0
11           孜然    斤   10.0
12           芝麻    斤    9.0
2           松花蛋    板    8.0
20         蟹黄粉丝    袋    7.0
15         三花淡奶    桶    7.0
14         烤肉

I m processing  /Users/user/processing_excel/2023.04/高频报货/571/哈西.xlsx
         items unit  price
6        黎红麻辣油    件  325.0
28       黎红麻辣油    件  325.0
12       黎红麻辣油    件  325.0
0          薄荷糖    件  220.0
26         薄荷糖    件  220.0
4       统厨蒜蓉辣酱    件   85.0
10          东古    桶   59.0
14          水塔    件   58.0
21         麻椒粒    斤   40.0
17       户户辣椒粉    袋   18.0
7          鲜花椒    袋   10.0
11         熟芝麻    斤   10.0
5          鲜花椒    袋   10.0
27         鲜花椒    袋   10.0
2          辣椒粉    斤   10.0
16         鲜花椒    袋   10.0
25         辣椒粉    斤   10.0
19         熟芝麻    斤   10.0
8          白芝麻    斤    9.0
9          白芝麻    斤    9.0
23         番茄酱    瓶    7.0
29          炼乳    瓶    7.0
18          炼乳    桶    7.0
13          炼乳    桶    7.0
22          淀粉    斤    4.5
20          火碱    斤    4.0
24          白糖    斤    4.0
1           火碱    斤    4.0
3           白糖    斤    4.0
15          白糖    斤    4.0
0       味好美蓝莓酱    件  365.0
1       纯真家乐鸡汁    件  335.0
2        黎红麻辣油    件  325.0
3          麻

I m processing  /Users/user/processing_excel/2023.04/高频报货/571/爱建.xlsx
         items unit  price
6        黎红麻辣油    件  325.0
0          薄荷糖    件  220.0
4       统厨蒜蓉辣酱    件   85.0
25         菜籽油    桶   75.0
22         圆泡椒    件   60.0
39         圆泡椒    件   60.0
23        东古酱油    桶   59.0
12         麻椒粒    斤   40.0
29       丘比沙拉汁    桶   40.0
40         麻椒粒    斤   40.0
18       丘比沙拉汁    桶   40.0
35         绿麻椒    斤   40.0
44       丘比沙拉汁    瓶   40.0
15        番茄沙司    桶   35.0
9          花生脆    袋   35.0
36         花生碎    袋   35.0
26       风球番茄酱    桶   35.0
17    味好美香甜沙拉酱    袋   25.0
10       户户辣椒粉    袋   18.0
33      安多夫松肉粉    瓶   16.0
14         煲汤袋    袋   13.0
7           麻酱    瓶   10.0
2          辣椒粉    斤   10.0
5          鲜花椒    袋   10.0
45         鲜花椒    袋   10.0
27         鲜花椒    袋   10.0
13       香甜沙拉酱    瓶    9.0
31        鲜味生抽    瓶    8.0
11       绿湖胡椒粉    袋    8.0
21          青豆    袋    8.0
37          炼乳    瓶    7.0
38        大粒花生    斤    7.0
42         番茄酱    瓶    7.0
47          

processing finished


In [10]:

# print(12 in datePickerList)
# print(datePickerList)